In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format

import copy
import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
import tensorflow as tf
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

# import scikitplot as skplt
# from sklearn.metrics import roc_auc_score
# import matplotlib.pyplot as plt

# # split dataset
# import splitfolders

# ignore warnings
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train.csv')
df

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6
...,...,...
718,719.PNG,9
719,720.PNG,6
720,721.PNG,2
721,722.PNG,8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  723 non-null    object
 1   label      723 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB


In [ ]:
df = df.astype({'label':'string'})

In [ ]:
label = df['label'].unique()
label

<StringArray>
['9', '4', '1', '6', '5', '8', '7', '2', '3', '0']
Length: 10, dtype: string

#데이터 이미지의 local adress와 label 값을 list에 저장

In [ ]:
import os
from glob import glob

def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
        
    # get label
    label_list.extend(df['label'])
                
    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    #print(img_path_list)
    
    return img_path_list

In [ ]:
all_img_path, all_label = get_train_data('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train')
test_img_path = get_test_data('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test')

In [ ]:
all_img_path[:5]


['/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/001.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/002.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/003.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/004.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/005.PNG']

In [ ]:
all_label[:5]

['9', '4', '1', '1', '6']

In [ ]:
test_img_path[:5]

['/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/001.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/002.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/003.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/004.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/005.PNG']

# Load Model

In [ ]:
pip install vit-keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=d5c41d3bf577fb4f14be8fc5599ecff3cdeb62764b69afd8d313b11ce6ca7aa6
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
from vit_keras import vit, utils
image_size= 224
vit = vit.vit_l32(
    image_size=image_size,
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False
)

In [ ]:
# resnet50 = tf.keras.applications.resnet50.ResNet50(
#     include_top=False,
#     weights='imagenet',
#     input_tensor=None,
#     input_shape=(224,224,3),
#     pooling=None
# )

### mobilenet

In [ ]:
#mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,  weights="imagenet",input_shape=(224,224,3,))


In [ ]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 32 #32

# Training 수
epochs = 20 # 20

# x_col = 'file_name'
# y_col = label

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 224
img_height = 224

# a = mobilenet_model.layers[-1].output
# a = resnet50.layers[-1].output
a = vit.layers[-1].output
# x = tf.keras.layers.GlobalAveragePooling2D()(a)
predictions = Dense(10, activation='softmax')(a)

# Input ~ Output 연결해주기
# DeepLearning = Model(inputs = mobilenet_model.input, outputs=predictions)
# DeepLearning = Model(inputs = resnet50.input, outputs=predictions)
DeepLearning = Model(inputs = vit.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
DeepLearning.compile(optimizer=
         SGD(lr=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc'])

In [ ]:
train_directory = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/'
test_directory  = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/'
model_directory = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/'
tensorboard_directory = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/' + 'tensorboard'



In [ ]:
# 사진을 제대로 불러오는지 확인
# import os
# import glob
# import cv2
# from google.colab.patches import cv2_imshow

# images = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/001.PNG'
# image = cv2.imread(images)
# cv2_imshow(image)

In [ ]:
DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

In [ ]:
# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    df,
    train_directory,
    x_col = 'file_name',
    y_col = 'label',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical", # softmax
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    df,
    train_directory,
    x_col = 'file_name',
    y_col = 'label',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode="categorical",
    subset = "validation")

# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_dataframe(
    df,
    test_directory,
    x_col = 'file_name',
    y_col = 'label',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')


# sigmoid / binary / binary_crossentropy
# softmax / categorical / categorical_crossentropy

Found 651 validated image filenames belonging to 10 classes.
Found 72 validated image filenames belonging to 10 classes.
Found 199 validated image filenames belonging to 10 classes.


In [ ]:
CP = ModelCheckpoint(filepath=model_directory+'vit-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_acc', verbose=1, save_best_only=True, mode='max')
TB = TensorBoard(log_dir=tensorboard_directory, write_graph=True, write_images=True)
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, TB, LR]

In [ ]:
DeepLearning.fit(
        TRAIN_GENERATOR,
        steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        epochs=50,
        callbacks=CALLBACK,
        shuffle=True, 
        validation_data=VALID_GENERATOR)

In [ ]:
DeepLearning.load_weights('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/vit-009-0.0687-1.0000.hdf5')


In [ ]:
TEST_Prediction = DeepLearning.predict_generator(TEST_GENERATOR, verbose=1)


7/7 [==============================] - 124s 20s/step


In [ ]:
TEST_Prediction[0:5]

array([[8.90629180e-03, 9.04709334e-04, 2.75244683e-01, 7.23407604e-03,
        8.85767513e-04, 1.34302641e-03, 8.18960034e-05, 6.71299458e-01,
        3.35385799e-02, 5.61540539e-04],
       [2.41260102e-04, 9.98397887e-01, 6.80613084e-05, 2.19953450e-04,
        9.48047727e-06, 1.53051093e-04, 9.04561530e-05, 2.52861209e-05,
        6.49580907e-04, 1.45032216e-04],
       [7.78360143e-02, 2.37289350e-03, 4.55139065e-03, 1.64939500e-02,
        2.31632101e-03, 1.21590525e-01, 3.15421261e-02, 4.17118147e-03,
        1.48812700e-02, 7.24244356e-01],
       [2.13582758e-02, 9.37883451e-04, 2.49578193e-01, 1.70517911e-03,
        1.31424365e-03, 1.28148298e-04, 2.47973553e-03, 7.14752138e-01,
        5.09095285e-03, 2.65527540e-03],
       [1.17103197e-03, 2.45738123e-03, 4.15876129e-05, 4.37713490e-04,
        1.59181212e-03, 9.85779567e-04, 9.91387784e-01, 1.13031722e-03,
        4.38129267e-04, 3.58520629e-04]], dtype=float32)

In [ ]:
TEST_Prediction.argmax(axis=1)

array([7, 1, 9, 7, 6, 1, 3, 6, 3, 1, 8, 1, 7, 1, 2, 3, 2, 0, 4, 0, 6, 4,
       8, 0, 8, 9, 2, 9, 1, 8, 0, 8, 6, 3, 3, 0, 7, 8, 9, 2, 5, 4, 3, 8,
       5, 6, 4, 9, 0, 3, 5, 4, 1, 5, 9, 3, 5, 6, 4, 0, 5, 6, 1, 5, 0, 9,
       0, 2, 0, 8, 3, 4, 4, 4, 6, 4, 4, 2, 2, 5, 7, 9, 9, 4, 3, 0, 1, 8,
       1, 8, 9, 0, 7, 6, 2, 5, 2, 6, 2, 7, 3, 5, 0, 9, 7, 6, 5, 9, 2, 0,
       7, 8, 4, 8, 8, 5, 4, 5, 1, 5, 4, 5, 1, 6, 4, 2, 9, 8, 6, 0, 8, 7,
       1, 8, 6, 3, 4, 6, 7, 7, 1, 8, 2, 5, 2, 6, 2, 6, 7, 6, 8, 3, 5, 7,
       8, 5, 6, 6, 6, 5, 6, 2, 3, 0, 5, 0, 4, 2, 5, 1, 9, 7, 1, 4, 3, 3,
       3, 8, 1, 4, 1, 4, 6, 7, 3, 9, 3, 2, 7, 2, 2, 7, 2, 8, 0, 1, 1, 9,
       0])

In [ ]:
Result = pd.DataFrame(TEST_Prediction)*100
Result

,0,1,2,3,4,5,6,7,8,9
0,0.89063,0.09047,27.52447,0.72341,0.08858,0.13430,0.00819,67.12994,3.35386,0.05615
1,0.02413,99.83979,0.00681,0.02200,0.00095,0.01531,0.00905,0.00253,0.06496,0.01450
2,7.78360,0.23729,0.45514,1.64939,0.23163,12.15905,3.15421,0.41712,1.48813,72.42444
3,2.13583,0.09379,24.95782,0.17052,0.13142,0.01281,0.24797,71.47521,0.50910,0.26553
4,0.11710,0.24574,0.00416,0.04377,0.15918,0.09858,99.13878,0.11303,0.04381,0.03585
...,...,...,...,...,...,...,...,...,...,...
194,91.16064,0.09825,4.39284,0.34490,0.45543,0.37923,1.68293,0.39072,0.13143,0.96363
195,0.17534,99.46820,0.09142,0.01105,0.01172,0.00307,0.13077,0.00235,0.09997,0.00610
196,0.14364,99.30487,0.07551,0.04061,0.00410,0.03960,0.01003,0.02041,0.32879,0.03245
197,0.12156,0.02389,0.04558,0.01063,0.01661,0.01793,0.02601,0.23365,0.03598,99.46815


In [ ]:
Result[0:5]


,0,1,2,3,4,5,6,7,8,9
0,0.89063,0.09047,27.52447,0.72341,0.08858,0.13430,0.00819,67.12994,3.35386,0.05615
1,0.02413,99.83979,0.00681,0.02200,0.00095,0.01531,0.00905,0.00253,0.06496,0.01450
2,7.78360,0.23729,0.45514,1.64939,0.23163,12.15905,3.15421,0.41712,1.48813,72.42444
3,2.13583,0.09379,24.95782,0.17052,0.13142,0.01281,0.24797,71.47521,0.50910,0.26553
4,0.11710,0.24574,0.00416,0.04377,0.15918,0.09858,99.13878,0.11303,0.04381,0.03585


#Submission file

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/sample_submission.csv')
submission 

,file_name,label
0,001.PNG,0
1,002.PNG,0
2,003.PNG,0
3,004.PNG,0
4,005.PNG,0
...,...,...
194,195.PNG,0
195,196.PNG,0
196,197.PNG,0
197,198.PNG,0


In [ ]:
submission['label'] = TEST_Prediction.argmax(axis=1)

In [ ]:
submission.head()

,file_name,label
0,001.PNG,7
1,002.PNG,1
2,003.PNG,9
3,004.PNG,7
4,005.PNG,6


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/submission/submission4.csv', index=False)


#epoch 50

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/sample_submission.csv')
submission 